In [1]:
import pandas as pd
import numpy as np


In [2]:
file = open('checkins.dat','r').readlines()
for ind,line in enumerate(file):
    file[ind] = file[ind].replace(' ', '')


In [3]:
out = open('checkins.csv', 'w')
for line in file:
    out.write(line)

In [4]:
data = pd.read_csv('checkins.csv', sep='|',low_memory=False)

In [5]:
data

,id,user_id,venue_id,latitude,longitude,created_at
0,---------+---------+----------+---------------...,NaN,NaN,NaN,NaN,NaN
1,984301,2041916.0,5222.0,NaN,NaN,2012-04-2117:39:01
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-2117:43:47
3,984315,1764391.0,5222.0,NaN,NaN,2012-04-2117:37:18
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-2117:43:43
...,...,...,...,...,...,...
1021848,928324,420125.0,36284.0,40.749824,-73.797634,2012-04-0804:59:13
1021849,928632,2118293.0,60.0,NaN,NaN,2012-04-0810:46:40
1021850,928744,2046000.0,4432.0,NaN,NaN,2012-04-0814:35:12
1021851,928900,97442.0,10936.0,40.740389,-73.983385,2012-04-0816:05:12


In [6]:
data.keys()

Index(['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at'], dtype='object')

In [7]:
data['latitude']

0                NaN
1                NaN
2          38.895112
3                NaN
4          33.800745
             ...    
1021848    40.749824
1021849          NaN
1021850          NaN
1021851    40.740389
1021852    33.990845
Name: latitude, Length: 1021853, dtype: float64

In [8]:
data.dropna(axis=0, inplace=True, subset=['latitude','longitude'])

In [9]:
data

,id,user_id,venue_id,latitude,longitude,created_at
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-2117:43:47
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-2117:43:43
8,984291,105054.0,5222.0,45.523452,-122.676207,2012-04-2117:39:22
10,984318,2146539.0,5222.0,40.764462,-111.904565,2012-04-2117:35:46
11,984232,93870.0,380645.0,33.448377,-112.074037,2012-04-2117:38:18
...,...,...,...,...,...,...
1021843,926487,1017859.0,60.0,35.772096,-78.638615,2012-04-0714:55:06
1021845,927215,174305.0,842313.0,28.320007,-80.607551,2012-04-0721:13:43
1021848,928324,420125.0,36284.0,40.749824,-73.797634,2012-04-0804:59:13
1021851,928900,97442.0,10936.0,40.740389,-73.983385,2012-04-0816:05:12


In [10]:
from sklearn.cluster import MeanShift

In [11]:
cluster = MeanShift(bandwidth=0.1)

In [12]:
cluster.fit(data[['latitude','longitude']][:100_000])

MeanShift(bandwidth=0.1)

In [14]:
cluster.predict(data[['latitude','longitude']][:100_000])

array([ 5,  7, 30, ..., 25, 19,  4], dtype=int64)

In [15]:
from collections import Counter
count = Counter(cluster.predict(data[['latitude','longitude']][:100_000]))

In [18]:
centers = []
for key in count:
    if count[key] >= 15:
        centers.append(cluster.cluster_centers_[key])
len(centers) 

624

In [22]:
offices = [[33.751277, -118.188740] ,
[25.867736, -80.324116],
[51.503016, -0.075479],
[52.378894, 4.885084],
[39.366487, 117.036146],
[-33.868457, 151.205134]]

In [23]:
def distance(pair_1,pair_2):
    import math
    return math.sqrt((pair_1[0]-pair_2[0])**2 + (pair_1[1]-pair_2[1])**2)

In [26]:
cent_dist = dict()
for val_cent in centers:
    min=100_000_000
    for val_off in offices:
        if distance(val_cent,val_off) < min:
            min = distance(val_cent,val_off)
    cent_dist[tuple(val_cent)] = min
sorted_tuple = sorted(cent_dist.items(), key=lambda x: x[1])
        

In [27]:
sorted_tuple

[((-33.86063042857143, 151.20477592857145), 0.007834758163107856),
 ((52.37296399032261, 4.892317222580647), 0.009353316185992226),
 ((25.8456722642857, -80.31889059642857), 0.022674066158385495),
 ((51.502991260887086, -0.12553728870967767), 0.05005829482278787),
 ((33.8098779552631, -118.14892380690813), 0.07084773242719973),
 ((25.7858124199675, -80.2179380368254), 0.13410903336184654),
 ((25.705349721052592, -80.28342873815798), 0.1674059642503429),
 ((26.010098249285683, -80.19999058571432), 0.18887596060185083),
 ((33.8883253427586, -118.04892817172427), 0.19577945647763628),
 ((33.87298601157018, -118.36209114655645), 0.21181053682436798),
 ((33.97257482142858, -118.16837066666663), 0.2222332907317907),
 ((26.138843786842077, -80.33434683684207), 0.2713007595066735),
 ((33.98393587403844, -118.00740497307689), 0.2949788868004569),
 ((26.120862658633104, -80.1589066802157), 0.3022701186924605),
 ((33.81730643390889, -117.891249170958), 0.30473050307840693),
 ((34.06039755458241, 